In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *
from math import pi,e
from numpy import linspace
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from bitarray import bitarray

importing ngsxfem-2.1.2504


In [2]:
def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [3]:
# physical parameters for linear elastic
mu  = 10
lam = 100

# parameters of DG method
order_u = 2
beta_u = 200
beta_u2 = 0

# parameter of ghost penalty
gamma_u1 = 0
gamma_u2 = 0



In [4]:
circle1 = 1/25-(x+0.75)**2-(y+0.25)**2
circle2 = 1/16-(x-0.5)**2-(y-0.2)**2
circle3 = 1-x**2*36 - y**2*9
f1 = IfPos(circle1-circle2,circle1,circle2)
levelset = IfPos(f1-circle3,f1,circle3)


In [5]:
u_x = x**2 + 1
u_y = y**2 + 1

In [6]:
# manufactured solution
exact_u = CF((u_x, u_y))

# strain tensor
epsilon_xx = u_x.Diff(x)
epsilon_yy = u_y.Diff(y) 
epsilon_xy = 0.5*(u_x.Diff(y) +  u_y.Diff(x))


# total stress tensor
sigma_xx = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_xx 
sigma_yy = lam*(epsilon_xx + epsilon_yy) + 2*mu*epsilon_yy 
sigma_xy = 2*mu*epsilon_xy


# 右端项 f_x, f_y
f_x = - (sigma_xx.Diff(x) + sigma_xy.Diff(y))
f_y = - (sigma_xy.Diff(x) + sigma_yy.Diff(y))

fe = CF((f_x, f_y))

uD = exact_u

In [7]:
# circle = SplineGeometry()
# circle.AddCircle((0,0),1/2)
# ngmesh = circle.GenerateMesh(maxh=h0, quad_dominated=quad_mesh)
# mesh = Mesh(ngmesh)
# Draw(mesh)

In [8]:
# Construct background mesh
# Geometry and Mesh
h0 = 1/32
quad_mesh = False
square = SplineGeometry()
square.AddRectangle((-1, -0.5), (1, 0.5), bc='outer')
ngmesh = square.GenerateMesh(maxh=h0, quad_dominated=quad_mesh)
mesh = Mesh(ngmesh)
# Draw(mesh)

# Higher order level set approximation
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=order_u, threshold=0.1,
                                      discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)
lsetp1 = lsetmeshadap.lset_p1

# DrawDC(lsetp1,-3.5,2.5,mesh)

# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
Draw(BitArrayCF(hasneg),mesh)
# Draw(BitArrayCF(hasif),mesh)

# facets used for stabilization:
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
interior_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)
in_surround_facets = GetElementsWithNeighborFacets(mesh,interior_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets") 
# Draw(BitArrayCF(in_surround_facets), mesh, "surrounding_facets") 


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [9]:
# print(help(mesh))

In [10]:
#Unfitted DG spaces
Uhbase = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for displacement
# U = Restrict(Uhbase,ci.GetElementsOfType(HASNEG))
U = Compress(Uhbase, GetDofsOfElements(Uhbase, ci.GetElementsOfType(HASNEG)))
u,v = U.TnT()

# Define the jumps and the averages
n = Normalize(grad(lsetp1)) # outer normal vector on the boundary
# n = 1.0 / Norm(grad(lsetp1)) * grad(lsetp1)
ne = specialcf.normal(mesh.dim) # normal vectors on faces
h = specialcf.mesh_size  

strain_u = Sym(Grad(u))
strain_v = Sym(Grad(v))
mean_stress_u = 0.5*(Stress(Sym(Grad(u)))+Stress(Sym(Grad(u.Other()))))*ne
mean_stress_v = 0.5*(Stress(Sym(Grad(v)))+Stress(Sym(Grad(v.Other()))))*ne
jump_u = u - u.Other()
jump_v = v - v.Other()

jump_du = Grad(u)*ne - Grad(u.Other())*ne
jump_dv = Grad(v)*ne - Grad(v.Other())*ne

# integration domains:
domega = dCut(lsetp1, NEG, definedonelements=hasneg, deformation=deformation)
dk = dCut(lsetp1, NEG, skeleton=True, definedonelements=interior_facets,
          deformation=deformation)
dsc = dCut(lsetp1, IF, definedonelements=hasif, deformation=deformation)
# dw = dCut(lsetp1, NEG, skeleton=True, definedonelements=ba_facets,
#           deformation=deformation)
dw = dFacetPatch(definedonelements=ba_facets, deformation=deformation)
df = dx(skeleton=True,definedonelements=ba_facets,deformation=deformation)
dsbar = ds(skeleton=True)

In [11]:
# main parts of Ah and lh
# stiffness matrix
Ah = BilinearForm(U)
Ah += 2*mu*InnerProduct(strain_u,strain_v)*domega + lam*div(u)*div(v)*domega \
            - (InnerProduct(mean_stress_u,jump_v) + InnerProduct(mean_stress_v,jump_u))*dk \
            - (InnerProduct(Stress(Sym(Grad(u)))*n,v) + InnerProduct(Stress(Sym(Grad(v)))*n,u))*dsc
Ah += - (InnerProduct(Stress(Sym(Grad(u)))*ne,v) + InnerProduct(Stress(Sym(Grad(v)))*ne,u))*dsbar

# r.h.s
lh = LinearForm(U) 
lh += fe*v*domega - InnerProduct(uD,Stress(Sym(Grad(v)))*n)*dsc - InnerProduct(uD,Stress(Sym(Grad(v)))*ne)*dsbar 


################################# Nitsche penalty terms #################################

# classical type
# Ah += beta_u/h*InnerProduct(jump_u,jump_v)*dk  # interior jump
# Ah += beta_u/h*InnerProduct(u,v)*ds 
# lh += beta_u/h*InnerProduct(uD,v)*ds

# cutFEM type
Ah += beta_u/h*(2*mu*InnerProduct(jump_u,jump_v)+lam*InnerProduct(jump_u,ne)*InnerProduct(jump_v,ne))*dk  # interior jump
Ah += beta_u/h*(2*mu*InnerProduct(u,v) + lam*InnerProduct(u,n)*InnerProduct(v,n))*dsc 
lh += beta_u/h*(2*mu*InnerProduct(uD,v) + lam*InnerProduct(uD,n)*InnerProduct(v,n))*dsc
Ah += beta_u/h*(2*mu*InnerProduct(u,v) + lam*InnerProduct(u,ne)*InnerProduct(v,ne))*dsbar
lh += beta_u/h*(2*mu*InnerProduct(uD,v) + lam*InnerProduct(uD,ne)*InnerProduct(v,ne))*dsbar

# divergence related type
# Ah += beta_u/h*InnerProduct(jump_u,jump_v)*dk  # interior jump
# Ah += beta_u2*h*(div(u)-div(u.Other()))*(div(v)-div(v.Other()))*dk
# Ah += beta_u/h*InnerProduct(u,v)*ds 
# lh += beta_u/h*InnerProduct(uD,v)*ds
# divu = uD[0].Diff(x)+uD[1].Diff(y)
# lh += beta_u2*h*divu*div(v)*ds
# lh += beta_u2/h*InnerProduct(Sym(Grad(uD)),Sym(Grad(v)))*ds

################################# ghost penalty terms #################################
Ah += gamma_u1 / (h**2) * jump_u * jump_v * dw
# Ah += gamma_u2 * InnerProduct(Sym(Grad(jump_u)),Sym(Grad(jump_v)))*dw
# Ah += gamma_u2 * (div(u)-div(u.Other()))*(div(v)-div(v.Other()))*dw


# Ah += gamma_u  * InnerProduct(Grad(u) - Grad(u.Other()),Grad(v) - Grad(v.Other())) * dw
# Ah += gamma_u * h * InnerProduct(Grad(u) - Grad(u.Other()),Grad(v) - Grad(v.Other())) * dw
# Ah += gamma_u * h * jump_du * jump_dv * df

# Assemble and solve 
Ah.Assemble()
lh.Assemble()
gfu = GridFunction(U)
gfu.vec.data = Ah.mat.Inverse() * lh.vec

In [12]:
error_u = sqrt(Integrate((gfu - exact_u)**2 * domega, mesh))
print(error_u)

1.6495583575684816e-05


In [13]:
# mask = IfPos(lsetp1,0,1)
mask = IfPos(levelset,0,1)

In [14]:
Draw(mask*gfu, mesh, deformation=deformation)
# Draw(exact_u,mesh,deformation=deformation)
Draw(mask*exact_u, mesh,deformation=deformation)
# Draw(mask*(gfu-exact_u),mesh,deformation=deformation)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [ ]:
deltau = CF((exact_u[0].Diff(x),exact_u[0].Diff(y),exact_u[1].Diff(x),exact_u[1].Diff(y)),dims=(2, 2)).Compile()

In [ ]:
grad_error_u = Grad(gfu)-deltau
error_u_H1 = sqrt(Integrate(InnerProduct(grad_error_u,grad_error_u)*domega, mesh))
print(error_u_H1)

In [ ]:
Draw(mask*Norm(grad_error_u),mesh,deformation=deformation)

In [ ]:
# gff = GridFunction(U)
# gff.Set(exact_u)
# grad_error_u = Grad(gfu - gff)
# error_u_H1 = sqrt(Integrate(InnerProduct(grad_error_u,grad_error_u)*domega, mesh))
# print(error_u_H1)

In [ ]:
# gff = GridFunction(Uhbase)
# gff.Interpolate(exact_u)
# err = gfu - gff
# strain_error = 0.5 * (Grad(err) + Grad(err).trans)
# # div_error = div(gfu) - div(gff)
# energy_error_sq = 0
# energy_error_sq += Integrate(InnerProduct(strain_error, strain_error)*domega,mesh)
# error_u_H1 = sqrt(energy_error_sq)
# print(error_u_H1)

In [ ]:
# gff = GridFunction(U)
# gff.Set(exact_u)
# grad_error_u = Grad(gfu - gff)
# error_u_H1 = sqrt(Integrate(InnerProduct(grad_error_u,grad_error_u)*  domega, mesh))
# print(error_u_H1)

In [ ]:
# DrawDC(lsetp1, gfu, (0,0), mesh, "uh", deformation=deformation)
# DrawDC(lsetp1, exact_u, (0,0), mesh, "uh", deformation=deformation)

In [ ]:
fesstress = MatrixValued(H1(mesh,order=order_u), symmetric=True)
gfstress = GridFunction(fesstress)

In [ ]:
# print(help(gfstress[0]))

In [ ]:
gfstress.Interpolate(Stress(Sym(deltau)))
# Draw(Norm(gfstress), mesh,deformation=deformation,min=0,max=500);
Draw(Norm(gfstress), mesh,deformation=deformation);

In [ ]:
gfstress.Interpolate(Stress(Sym(Grad(gff))))
Draw (Norm(gfstress), mesh,deformation=deformation);

In [ ]:
gfstress.Interpolate(Stress(Sym(Grad(gff-gfu))))
Draw (mask*Norm(gfstress[3]), mesh,deformation=deformation);

In [ ]:
gfstress.Interpolate(Stress(Sym(Grad(gfu)-deltau)))
Draw (mask*Norm(gfstress), mesh,min=0,max=0.1);
Draw (mask*Norm(gfstress), mesh,deformation=deformation);

In [ ]:
kappaminus = CutRatioGF(ci)
kappaminus_values = kappaminus.vec.FV().NumPy()
# positive_values = []
# positive_indices = []
# for v,ind in enumerate(kappaminus_values):
#     positive_values.append(v)
#     positive_indices.append(ind)
positive_values = [v for v in kappaminus_values if v > 0]
if positive_values:
    min_value_pythonic = min(positive_values)
    print(f"The smallest cut ratio is: {min_value_pythonic:.2e}")
else:
    print("There are no cut elements.")

In [ ]:
# positive_indices = np.array([v for v in range(len(kappaminus_values)) if kappaminus_values[v] > 0])
positive_indices = np.where(kappaminus_values > 0)[0]
# print(len(positive_values))
sorted_value_indices_in_positive_array = np.argsort(positive_values)
Ke = 10
smallest_K_indices_in_positive_array = sorted_value_indices_in_positive_array[:Ke]
smallest_K_original_indices = positive_indices[smallest_K_indices_in_positive_array]
min_K_values = kappaminus_values[smallest_K_original_indices]
print(f"最小的 {Ke} 个切割单元的比率: {min_K_values}")
print(f"它们在原始数组中的索引: {smallest_K_original_indices}")
print("-" * 30)

# visualize
smallest_K = BitArray(mesh.ne)
smallest_K.Clear()
for i in smallest_K_original_indices:
    smallest_K.Set(i)
Draw(BitArrayCF(smallest_K),mesh)

In [ ]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds

# 计算最大奇异值
rows,cols,vals = Ah.mat.COO()
A = sp.csr_matrix((vals,(rows,cols)))
condition_number = np.linalg.cond(A.todense())
# _, s_max,_ = svds(A, k=1, which='LM')   # largest magnitude

# # # 计算最小奇异值
# _, s_min,_ = svds(A, k=1, which='SM')   # smallest magnitude

# condition_number = s_max[0] / s_min[0]
print(f"{condition_number:.2e}")


In [ ]:
# Ah = BilinearForm(U)
# # Ae
# Ah += 2*mu*InnerProduct(strain_u,strain_v)*domega + lam*div(u)*div(v)*domega \
#         - (InnerProduct(mean_stress_u,jump_v) + InnerProduct(mean_stress_v,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dk \
#         - (InnerProduct(Stress(Sym(Grad(u)))*n,v) + InnerProduct(Stress(Sym(Grad(v)))*n,u) - beta_u/h*InnerProduct(u,v))*ds
# # order=1 i_s 
# Ah += gamma_u * h * ((Grad(u) - Grad(u.Other()))*ne) * ((Grad(v) - Grad(v.Other()))*ne) * dw
# # Ah += gamma_u * h * InnerProduct(Grad(u) - Grad(u.Other()),Grad(v) - Grad(v.Other())) * dw
# # Ah += gamma_u * h * jump_du * jump_dv * dw

# Ah.Assemble()

# # r.h.s
# lh = LinearForm(U) 
# lh += fe*v*domega - InnerProduct(uD,Stress(Sym(Grad(v)))*n)*ds + beta_u/h*uD*v*ds
# lh.Assemble()

# gfu = GridFunction(U)
# gfu.vec.data = Ah.mat.Inverse() * lh.vec